In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\cleaned_data.csv")
data

,UTC_Time,Date,Region,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,Solar_Gen,CO2_Factor_Coal,CO2_Factor_Gas,CO2_Emissions_Coal,CO2_Emissions_Gas,CO2_Total_Emissions,CO2_Intensity_Generated,CO2_Intensity_Consumed
0,2018-07-01 08:00:00,2018-07-01,CAL,1,26061.0,17023.0,-6508.0,25443.0,7.0,7675.0,2267.0,2560.0,0.0,2.23,0.93,7.07,3221.51,3373.68,0.44,0.38
1,2018-07-01 09:00:00,2018-07-01,CAL,2,24604.0,15411.0,-6710.0,24029.0,9.0,5879.0,2261.0,2584.0,0.0,2.23,0.93,9.09,2467.84,2623.17,0.37,0.33
2,2018-07-01 10:00:00,2018-07-01,CAL,3,23554.0,14680.0,-6840.0,22984.0,6.0,5635.0,2258.0,2506.0,0.0,2.23,0.92,6.06,2362.67,2511.83,0.38,0.33
3,2018-07-01 11:00:00,2018-07-01,CAL,4,22843.0,14362.0,-6843.0,22343.0,8.0,5405.0,2257.0,2446.0,0.0,2.23,0.93,8.08,2268.50,2419.01,0.37,0.32
4,2018-07-01 12:00:00,2018-07-01,CAL,5,22538.0,14155.0,-6928.0,22108.0,8.0,5592.0,2260.0,2609.0,0.0,2.23,0.93,8.08,2350.03,2498.24,0.39,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525469,2023-07-01 01:00:00,2023-06-30,TEX,20,73862.0,73920.0,58.0,75075.0,11013.0,36589.0,4642.0,115.0,2237.0,2.33,0.91,11662.81,15069.97,26833.94,0.80,0.80
525470,2023-07-01 02:00:00,2023-06-30,TEX,21,70570.0,70337.0,-232.0,71706.0,10632.0,36722.0,4643.0,133.0,264.0,2.34,0.91,11264.12,15132.49,26484.94,0.83,0.83
525471,2023-07-01 03:00:00,2023-06-30,TEX,22,67978.0,67953.0,-25.0,69197.0,10592.0,35912.0,4643.0,76.0,0.0,2.33,0.91,11217.49,14802.15,26102.08,0.85,0.85
525472,2023-07-01 04:00:00,2023-06-30,TEX,23,64727.0,64768.0,42.0,65551.0,10055.0,32700.0,4645.0,10.0,0.0,2.33,0.91,10637.39,13466.76,24188.75,0.82,0.82


In [4]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Create seasonal and time-based features
data['Month'] = data['Date'].dt.month                           # 1 to 12
data['DayOfWeek'] = data['Date'].dt.dayofweek                   # 0 = Monday, ..., 6 = Sunday
data['Is_Weekend'] = data['DayOfWeek'].isin([5, 6]).astype(int) # 1 if Saturday or Sunday
data['DayOfYear'] = data['Date'].dt.dayofyear                   # 1 to 365 (or 366)
data['WeekOfYear'] = data['Date'].dt.isocalendar().week         # 1 to 52 (or 53)

# (Optional) Check your new columns
print(data[['Date', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear']].head())

        Date  Month  DayOfWeek  Is_Weekend  DayOfYear  WeekOfYear
0 2018-07-01      7          6           1        182          26
1 2018-07-01      7          6           1        182          26
2 2018-07-01      7          6           1        182          26
3 2018-07-01      7          6           1        182          26
4 2018-07-01      7          6           1        182          26


In [5]:
# Ensure the 'Date' column is in datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort the data by date just in case
data = data.sort_values(by='Date').reset_index(drop=True)

# Create a complete date range from start to end
full_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='H')  # Change freq to 'D' for daily data

# Find missing dates
missing_dates = full_range.difference(data['Date'])

# Show result
if missing_dates.empty:
    print("✅ No missing timestamps — your data is continuous.")
else:
    print(f"❌ {len(missing_dates)} missing timestamps found.")
    print("Sample missing dates:")
    print(missing_dates[:10])  # show first 10

❌ 41975 missing timestamps found.
Sample missing dates:
DatetimeIndex(['2018-07-01 01:00:00', '2018-07-01 02:00:00',
               '2018-07-01 03:00:00', '2018-07-01 04:00:00',
               '2018-07-01 05:00:00', '2018-07-01 06:00:00',
               '2018-07-01 07:00:00', '2018-07-01 08:00:00',
               '2018-07-01 09:00:00', '2018-07-01 10:00:00'],
              dtype='datetime64[ns]', freq=None)


In [6]:
import pandas as pd

# Load your dataset
data['Date'] = pd.to_datetime(data['Date'])

# Create an empty list to collect per-region reindexed data
region_dfs = []

# Get the list of unique regions
regions = data['Region'].unique()

# Loop through each region
for region in regions:
    # Filter data for that region
    region_data = data[data['Region'] == region].copy()

    # Generate complete hourly range for that region
    full_range = pd.date_range(start=region_data['Date'].min(), end=region_data['Date'].max(), freq='H')

    # Group and average numeric values for duplicate timestamps
    numeric_cols = region_data.select_dtypes(include='number').columns
    grouped = region_data.groupby('Date')[numeric_cols].mean().reset_index()

    # Reindex to full hourly timestamps
    reindexed = grouped.set_index('Date').reindex(full_range)
    reindexed.index.name = 'Date'

    # Fill missing region label
    reindexed['Region'] = region

    # Append to the list
    region_dfs.append(reindexed.reset_index())

# Combine all region DataFrames
data_full = pd.concat(region_dfs, ignore_index=True)

# Optional: Sort by Region and Date
data_full = data_full.sort_values(by=['Region', 'Date'])

# Show result
print("✅ Data successfully reindexed per region with missing hours filled as NaN.")
print(data_full.head())

✅ Data successfully reindexed per region with missing hours filled as NaN.
                 Date  Hour    Demand  Net_Generation  Total_Interchange  \
0 2018-07-01 00:00:00  12.5  27455.25    19785.416667       -6089.333333   
1 2018-07-01 01:00:00   NaN       NaN             NaN                NaN   
2 2018-07-01 02:00:00   NaN       NaN             NaN                NaN   
3 2018-07-01 03:00:00   NaN       NaN             NaN                NaN   
4 2018-07-01 04:00:00   NaN       NaN             NaN                NaN   

   Forecasted_Demand  Coal_Gen      Gas_Gen  Nuclear_Gen  Hydro_Gen  ...  \
0       26689.791667  8.291667  7114.541667      2261.25    2735.25  ...   
1                NaN       NaN          NaN          NaN        NaN  ...   
2                NaN       NaN          NaN          NaN        NaN  ...   
3                NaN       NaN          NaN          NaN        NaN  ...   
4                NaN       NaN          NaN          NaN        NaN  ...   

   CO2_Emis

In [7]:
data_full.interpolate(method='linear', inplace=True)

In [8]:
data = data_full.copy()

In [9]:
# Ensure the 'Date' column is in datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort the data by date just in case
data = data.sort_values(by='Date').reset_index(drop=True)

# Create a complete date range from start to end
full_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='H')  # Change freq to 'D' for daily data

# Find missing dates
missing_dates = full_range.difference(data['Date'])

# Show result
if missing_dates.empty:
    print("✅ No missing timestamps — your data is continuous.")
else:
    print(f"❌ {len(missing_dates)} missing timestamps found.")
    print("Sample missing dates:")
    print(missing_dates[:10])  # show first 10

✅ No missing timestamps — your data is continuous.


In [10]:
def assign_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

data['Season'] = data['Month'].apply(assign_season)
data = pd.get_dummies(data, columns=['Season'])  # One-hot encode Season
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Month,DayOfWeek,Is_Weekend,DayOfYear,WeekOfYear,Region,Season_Autumn,Season_Spring,Season_Summer,Season_Winter
0,2018-07-01,12.5,27455.250000,19785.416667,-6089.333333,26689.791667,8.291667,7114.541667,2261.250000,2735.250000,...,7.0,6.0,1.0,182.0,26.0,CAL,False,False,True,False
1,2018-07-01,12.5,114937.291667,118965.958333,3982.875000,114216.583333,38975.375000,38177.250000,33496.083333,2257.041667,...,7.0,6.0,1.0,182.0,26.0,MIDA,False,False,True,False
2,2018-07-01,12.5,17965.500000,15979.708333,-1985.791667,17863.750000,342.833333,9701.791667,3946.250000,862.166667,...,7.0,6.0,1.0,182.0,26.0,NE,False,False,True,False
3,2018-07-01,12.5,30623.583333,29623.416667,-1018.833333,28525.750000,3199.208333,21023.250000,3779.500000,31.291667,...,7.0,6.0,1.0,182.0,26.0,FLA,False,False,True,False
4,2018-07-01,12.5,39135.541667,43793.625000,9090.333333,37734.000000,9165.000000,7112.875000,986.375000,16349.708333,...,7.0,6.0,1.0,182.0,26.0,NW,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525223,2023-06-30,12.5,36658.541667,36016.250000,-641.125000,35350.541667,2432.458333,23954.958333,3683.875000,34.833333,...,6.0,4.0,0.0,181.0,26.0,FLA,False,False,True,False
525224,2023-06-30,12.5,40239.916667,40975.166667,735.333333,42000.375000,13035.541667,13727.625000,1968.833333,1565.833333,...,6.0,4.0,0.0,181.0,26.0,CENT,False,False,True,False
525225,2023-06-30,12.5,29410.166667,29558.875000,144.708333,30887.791667,7983.583333,6982.208333,11876.166667,368.083333,...,6.0,4.0,0.0,181.0,26.0,CAR,False,False,True,False
525226,2023-06-30,12.5,14294.583333,11865.291667,-2429.291667,14351.250000,0.166667,7797.666667,2040.500000,1186.166667,...,6.0,4.0,0.0,181.0,26.0,NE,False,False,True,False


In [11]:
data['Demand_Prev_Hour'] = data['Demand'].shift(1)
data['Demand_Yesterday_Same_Hour'] = data['Demand'].shift(24)
data['Demand_Last_Week_Same_Hour'] = data['Demand'].shift(168)

In [12]:
data.columns.to_list()

['Date',
 'Hour',
 'Demand',
 'Net_Generation',
 'Total_Interchange',
 'Forecasted_Demand',
 'Coal_Gen',
 'Gas_Gen',
 'Nuclear_Gen',
 'Hydro_Gen',
 'Solar_Gen',
 'CO2_Factor_Coal',
 'CO2_Factor_Gas',
 'CO2_Emissions_Coal',
 'CO2_Emissions_Gas',
 'CO2_Total_Emissions',
 'CO2_Intensity_Generated',
 'CO2_Intensity_Consumed',
 'Month',
 'DayOfWeek',
 'Is_Weekend',
 'DayOfYear',
 'WeekOfYear',
 'Region',
 'Season_Autumn',
 'Season_Spring',
 'Season_Summer',
 'Season_Winter',
 'Demand_Prev_Hour',
 'Demand_Yesterday_Same_Hour',
 'Demand_Last_Week_Same_Hour']

In [13]:
data['Emission_Prev_Hour'] = data['CO2_Total_Emissions'].shift(1)
data['Emission_Yesterday_Same_Hour'] = data['CO2_Total_Emissions'].shift(24)
data['Emission_Last_Week_Same_Hour'] = data['CO2_Total_Emissions'].shift(168)

In [14]:
data['Rolling_Mean_3H'] = data['Demand'].rolling(window=3).mean()
data['Rolling_Mean_24H'] = data['Demand'].rolling(window=24).mean()

data['Rolling_Mean_3H_Emission'] = data['CO2_Total_Emissions'].rolling(window=3).mean()
data['Rolling_Mean_24H_Emission'] = data['CO2_Total_Emissions'].rolling(window=24).mean()

# ========= 4. Energy Mix Ratios =========
data['Total_Gen'] = data[['Coal_Gen', 'Gas_Gen', 'Nuclear_Gen', 'Hydro_Gen', 'Solar_Gen']].sum(axis=1)

data['Renewable_Pct'] = ((data['Hydro_Gen'] + data['Solar_Gen']) / data['Total_Gen']) * 100
data['Fossil_Pct'] = ((data['Coal_Gen'] + data['Gas_Gen']) / data['Total_Gen']) * 100

data['Renewable_Demand'] = data['Renewable_Pct'] * data['Demand']
data['Fossil_Demand'] = data['Fossil_Pct'] * data['Demand']

# ========= 5. Final Cleanup =========
# Drop rows with NaNs created from rolling or lag
data = data.dropna().reset_index(drop=True)
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Emission_Last_Week_Same_Hour,Rolling_Mean_3H,Rolling_Mean_24H,Rolling_Mean_3H_Emission,Rolling_Mean_24H_Emission,Total_Gen,Renewable_Pct,Fossil_Pct,Renewable_Demand,Fossil_Demand
0,2018-07-01 16:00:00,12.500000,32668.750000,31610.944444,-1129.861111,30188.944444,3395.680556,22638.305556,3774.250000,31.569444,...,3161.941667,33690.347222,42616.980698,10145.391898,17120.063112,30126.555556,1.056607,86.415409,3.451805e+04,2.823083e+06
1,2018-07-01 16:00:00,12.666667,98191.374396,95908.335749,-3872.722222,101822.126812,47810.253623,29087.126812,11639.043478,1224.783816,...,54321.367083,56791.944243,45357.741958,29504.802506,19165.902579,89860.721014,1.473722,85.573963,1.447068e+05,8.402625e+06
2,2018-07-01 17:00:00,12.500000,31560.368056,33623.656250,2208.244792,17062.499167,9879.272569,16289.060764,5835.713542,1332.260417,...,4274.585833,54140.164151,45476.604516,30119.971123,19707.434642,33652.527778,4.898535,77.760380,1.545996e+05,2.454146e+06
3,2018-07-01 17:00:00,12.500000,28976.041667,21138.303819,-6528.677083,28386.220486,8.763889,7725.951389,2261.072917,3089.741319,...,11635.019583,52909.261373,42604.509495,27096.581638,17297.042357,17521.203125,42.950332,44.144887,1.244531e+06,1.279144e+06
4,2018-07-01 17:00:00,12.500000,8551.005208,9138.222222,586.211806,8408.137153,1511.822917,3661.506944,2692.883681,658.407986,...,12711.690000,23029.138310,41663.796026,7635.595110,16940.597026,8965.375000,12.260072,57.703441,1.048359e+05,4.934224e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-30 00:00:00,12.500000,36658.541667,36016.250000,-641.125000,35350.541667,2432.458333,23954.958333,3683.875000,34.833333,...,11992.780590,53861.175000,43957.871934,19576.309167,16891.339123,32352.291667,7.050505,81.562743,2.584612e+05,2.989971e+06
525056,2023-06-30 00:00:00,12.500000,40239.916667,40975.166667,735.333333,42000.375000,13035.541667,13727.625000,1968.833333,1565.833333,...,2952.594201,59835.236111,44972.352572,24390.933750,17500.097977,30351.791667,5.336725,88.176563,2.147494e+05,3.548218e+06
525057,2023-06-30 00:00:00,12.500000,29410.166667,29558.875000,144.708333,30887.791667,7983.583333,6982.208333,11876.166667,368.083333,...,38830.065660,35436.208333,43496.255639,14363.882778,17067.607339,28434.000000,5.599077,52.633438,1.646698e+05,1.547958e+06
525058,2023-06-30 00:00:00,12.500000,14294.583333,11865.291667,-2429.291667,14351.250000,0.166667,7797.666667,2040.500000,1186.166667,...,6178.587465,27981.555556,43495.837236,11441.087083,17068.228162,11185.333333,12.042556,69.714805,1.721433e+05,9.965441e+05


In [15]:
data.dtypes

Date                            datetime64[ns]
Hour                                   float64
Demand                                 float64
Net_Generation                         float64
Total_Interchange                      float64
Forecasted_Demand                      float64
Coal_Gen                               float64
Gas_Gen                                float64
Nuclear_Gen                            float64
Hydro_Gen                              float64
Solar_Gen                              float64
CO2_Factor_Coal                        float64
CO2_Factor_Gas                         float64
CO2_Emissions_Coal                     float64
CO2_Emissions_Gas                      float64
CO2_Total_Emissions                    float64
CO2_Intensity_Generated                float64
CO2_Intensity_Consumed                 float64
Month                                  float64
DayOfWeek                              float64
Is_Weekend                             float64
DayOfYear    

In [16]:
# Convert all boolean columns to integers (True → 1, False → 0)
bool_cols = data.select_dtypes(include='bool').columns
data[bool_cols] = data[bool_cols].astype(int)
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Emission_Last_Week_Same_Hour,Rolling_Mean_3H,Rolling_Mean_24H,Rolling_Mean_3H_Emission,Rolling_Mean_24H_Emission,Total_Gen,Renewable_Pct,Fossil_Pct,Renewable_Demand,Fossil_Demand
0,2018-07-01 16:00:00,12.500000,32668.750000,31610.944444,-1129.861111,30188.944444,3395.680556,22638.305556,3774.250000,31.569444,...,3161.941667,33690.347222,42616.980698,10145.391898,17120.063112,30126.555556,1.056607,86.415409,3.451805e+04,2.823083e+06
1,2018-07-01 16:00:00,12.666667,98191.374396,95908.335749,-3872.722222,101822.126812,47810.253623,29087.126812,11639.043478,1224.783816,...,54321.367083,56791.944243,45357.741958,29504.802506,19165.902579,89860.721014,1.473722,85.573963,1.447068e+05,8.402625e+06
2,2018-07-01 17:00:00,12.500000,31560.368056,33623.656250,2208.244792,17062.499167,9879.272569,16289.060764,5835.713542,1332.260417,...,4274.585833,54140.164151,45476.604516,30119.971123,19707.434642,33652.527778,4.898535,77.760380,1.545996e+05,2.454146e+06
3,2018-07-01 17:00:00,12.500000,28976.041667,21138.303819,-6528.677083,28386.220486,8.763889,7725.951389,2261.072917,3089.741319,...,11635.019583,52909.261373,42604.509495,27096.581638,17297.042357,17521.203125,42.950332,44.144887,1.244531e+06,1.279144e+06
4,2018-07-01 17:00:00,12.500000,8551.005208,9138.222222,586.211806,8408.137153,1511.822917,3661.506944,2692.883681,658.407986,...,12711.690000,23029.138310,41663.796026,7635.595110,16940.597026,8965.375000,12.260072,57.703441,1.048359e+05,4.934224e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-30 00:00:00,12.500000,36658.541667,36016.250000,-641.125000,35350.541667,2432.458333,23954.958333,3683.875000,34.833333,...,11992.780590,53861.175000,43957.871934,19576.309167,16891.339123,32352.291667,7.050505,81.562743,2.584612e+05,2.989971e+06
525056,2023-06-30 00:00:00,12.500000,40239.916667,40975.166667,735.333333,42000.375000,13035.541667,13727.625000,1968.833333,1565.833333,...,2952.594201,59835.236111,44972.352572,24390.933750,17500.097977,30351.791667,5.336725,88.176563,2.147494e+05,3.548218e+06
525057,2023-06-30 00:00:00,12.500000,29410.166667,29558.875000,144.708333,30887.791667,7983.583333,6982.208333,11876.166667,368.083333,...,38830.065660,35436.208333,43496.255639,14363.882778,17067.607339,28434.000000,5.599077,52.633438,1.646698e+05,1.547958e+06
525058,2023-06-30 00:00:00,12.500000,14294.583333,11865.291667,-2429.291667,14351.250000,0.166667,7797.666667,2040.500000,1186.166667,...,6178.587465,27981.555556,43495.837236,11441.087083,17068.228162,11185.333333,12.042556,69.714805,1.721433e+05,9.965441e+05


In [17]:
data['Demand_diff1'] = data.groupby('Region')['Demand'].diff(1)
data['Demand_diff24'] = data.groupby('Region')['Demand'].diff(24)

In [18]:
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Rolling_Mean_24H,Rolling_Mean_3H_Emission,Rolling_Mean_24H_Emission,Total_Gen,Renewable_Pct,Fossil_Pct,Renewable_Demand,Fossil_Demand,Demand_diff1,Demand_diff24
0,2018-07-01 16:00:00,12.500000,32668.750000,31610.944444,-1129.861111,30188.944444,3395.680556,22638.305556,3774.250000,31.569444,...,42616.980698,10145.391898,17120.063112,30126.555556,1.056607,86.415409,3.451805e+04,2.823083e+06,NaN,NaN
1,2018-07-01 16:00:00,12.666667,98191.374396,95908.335749,-3872.722222,101822.126812,47810.253623,29087.126812,11639.043478,1224.783816,...,45357.741958,29504.802506,19165.902579,89860.721014,1.473722,85.573963,1.447068e+05,8.402625e+06,NaN,NaN
2,2018-07-01 17:00:00,12.500000,31560.368056,33623.656250,2208.244792,17062.499167,9879.272569,16289.060764,5835.713542,1332.260417,...,45476.604516,30119.971123,19707.434642,33652.527778,4.898535,77.760380,1.545996e+05,2.454146e+06,NaN,NaN
3,2018-07-01 17:00:00,12.500000,28976.041667,21138.303819,-6528.677083,28386.220486,8.763889,7725.951389,2261.072917,3089.741319,...,42604.509495,27096.581638,17297.042357,17521.203125,42.950332,44.144887,1.244531e+06,1.279144e+06,NaN,NaN
4,2018-07-01 17:00:00,12.500000,8551.005208,9138.222222,586.211806,8408.137153,1511.822917,3661.506944,2692.883681,658.407986,...,41663.796026,7635.595110,16940.597026,8965.375000,12.260072,57.703441,1.048359e+05,4.934224e+05,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-30 00:00:00,12.500000,36658.541667,36016.250000,-641.125000,35350.541667,2432.458333,23954.958333,3683.875000,34.833333,...,43957.871934,19576.309167,16891.339123,32352.291667,7.050505,81.562743,2.584612e+05,2.989971e+06,-58.500000,-1404.000000
525056,2023-06-30 00:00:00,12.500000,40239.916667,40975.166667,735.333333,42000.375000,13035.541667,13727.625000,1968.833333,1565.833333,...,44972.352572,24390.933750,17500.097977,30351.791667,5.336725,88.176563,2.147494e+05,3.548218e+06,-78.631944,-1887.166667
525057,2023-06-30 00:00:00,12.500000,29410.166667,29558.875000,144.708333,30887.791667,7983.583333,6982.208333,11876.166667,368.083333,...,43496.255639,14363.882778,17067.607339,28434.000000,5.599077,52.633438,1.646698e+05,1.547958e+06,-5.975694,-143.416667
525058,2023-06-30 00:00:00,12.500000,14294.583333,11865.291667,-2429.291667,14351.250000,0.166667,7797.666667,2040.500000,1186.166667,...,43495.837236,11441.087083,17068.228162,11185.333333,12.042556,69.714805,1.721433e+05,9.965441e+05,-5.020833,-120.500000


In [19]:
data = data.sort_values(['Region', 'Date'])
data = data.groupby('Region').apply(lambda group: group.ffill().bfill()).reset_index(drop=True)
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Rolling_Mean_24H,Rolling_Mean_3H_Emission,Rolling_Mean_24H_Emission,Total_Gen,Renewable_Pct,Fossil_Pct,Renewable_Demand,Fossil_Demand,Demand_diff1,Demand_diff24
0,2018-07-01 17:00:00,12.5,28976.041667,21138.303819,-6528.677083,28386.220486,8.763889,7725.951389,2261.072917,3089.741319,...,42604.509495,27096.581638,17297.042357,17521.203125,42.950332,44.144887,1.244531e+06,1.279144e+06,89.458333,172.875000
1,2018-07-01 18:00:00,12.5,29065.500000,21217.885417,-6554.520833,28486.010417,8.791667,7761.916667,2261.062500,3110.593750,...,41847.194542,8244.811736,17052.778490,17577.072917,42.926954,44.209342,1.247693e+06,1.284967e+06,89.458333,172.875000
2,2018-07-01 19:00:00,12.5,29154.958333,21297.467014,-6580.364583,28585.800347,8.819444,7797.881944,2261.052083,3131.446181,...,45629.696838,24266.185920,19327.269156,17632.942708,42.903725,44.273389,1.250856e+06,1.290789e+06,89.458333,172.875000
3,2018-07-01 20:00:00,12.5,29244.416667,21377.048611,-6606.208333,28685.590278,8.847222,7833.847222,2261.041667,3152.298611,...,48609.445555,6436.826227,20904.933236,17688.812500,42.880642,44.337032,1.254019e+06,1.296611e+06,89.458333,172.875000
4,2018-07-01 21:00:00,12.5,29333.875000,21456.630208,-6632.052083,28785.380208,8.875000,7869.812500,2261.031250,3173.151042,...,49142.567227,6326.084242,21205.564127,17744.682292,42.857705,44.400274,1.257183e+06,1.302432e+06,89.458333,172.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-29 20:00:00,12.5,64967.777778,65025.979167,58.041667,65732.083333,8989.486111,27552.909722,4647.180556,32.895833,...,49409.848920,30361.663657,19267.172606,46077.069444,10.607213,79.307118,6.891271e+05,5.152407e+06,-65.642361,-1360.548611
525056,2023-06-29 21:00:00,12.5,64902.135417,64963.473958,61.177083,65699.302083,8993.729167,27593.401042,4647.291667,33.223958,...,46137.266045,13730.127535,18243.585852,46106.208333,10.566444,79.354021,6.857847e+05,5.150245e+06,-65.642361,-1414.265625
525057,2023-06-29 22:00:00,12.5,64836.493056,64900.968750,64.312500,65666.520833,8997.972222,27633.892361,4647.402778,33.552083,...,45454.091215,12961.713287,16905.630206,46135.347222,10.525725,79.400865,6.824511e+05,5.148074e+06,-65.642361,-1467.982639
525058,2023-06-29 23:00:00,12.5,64770.850694,64838.463542,67.447917,65633.739583,9002.215278,27674.383681,4647.513889,33.880208,...,39779.803738,15939.368576,14414.640501,46164.486111,10.485058,79.447649,6.791261e+05,5.145892e+06,-65.642361,-1521.699653


In [20]:
data = data.round(3)
bool_cols = data.select_dtypes(include='bool').columns
data[bool_cols] = data[bool_cols].astype(int)
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Rolling_Mean_24H,Rolling_Mean_3H_Emission,Rolling_Mean_24H_Emission,Total_Gen,Renewable_Pct,Fossil_Pct,Renewable_Demand,Fossil_Demand,Demand_diff1,Demand_diff24
0,2018-07-01 17:00:00,12.5,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,42604.509,27096.582,17297.042,17521.203,42.950,44.145,1244530.613,1279144.078,89.458,172.875
1,2018-07-01 18:00:00,12.5,29065.500,21217.885,-6554.521,28486.010,8.792,7761.917,2261.062,3110.594,...,41847.195,8244.812,17052.778,17577.073,42.927,44.209,1247693.394,1284966.639,89.458,172.875
2,2018-07-01 19:00:00,12.5,29154.958,21297.467,-6580.365,28585.800,8.819,7797.882,2261.052,3131.446,...,45629.697,24266.186,19327.269,17632.943,42.904,44.273,1250856.313,1290788.823,89.458,172.875
3,2018-07-01 20:00:00,12.5,29244.417,21377.049,-6606.208,28685.590,8.847,7833.847,2261.042,3152.299,...,48609.446,6436.826,20904.933,17688.812,42.881,44.337,1254019.366,1296610.635,89.458,172.875
4,2018-07-01 21:00:00,12.5,29333.875,21456.630,-6632.052,28785.380,8.875,7869.812,2261.031,3173.151,...,49142.567,6326.084,21205.564,17744.682,42.858,44.400,1257182.553,1302432.078,89.458,172.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-29 20:00:00,12.5,64967.778,65025.979,58.042,65732.083,8989.486,27552.910,4647.181,32.896,...,49409.849,30361.664,19267.173,46077.069,10.607,79.307,689127.079,5152407.218,-65.642,-1360.549
525056,2023-06-29 21:00:00,12.5,64902.135,64963.474,61.177,65699.302,8993.729,27593.401,4647.292,33.224,...,46137.266,13730.128,18243.586,46106.208,10.566,79.354,685784.748,5150245.412,-65.642,-1414.266
525057,2023-06-29 22:00:00,12.5,64836.493,64900.969,64.312,65666.521,8997.972,27633.892,4647.403,33.552,...,45454.091,12961.713,16905.630,46135.347,10.526,79.401,682451.108,5148073.607,-65.642,-1467.983
525058,2023-06-29 23:00:00,12.5,64770.851,64838.464,67.448,65633.740,9002.215,27674.384,4647.514,33.880,...,39779.804,15939.369,14414.641,46164.486,10.485,79.448,679126.143,5145891.821,-65.642,-1521.700


In [46]:
#data.to_csv(r"C:\Users\rosem\Downloads\Paper_works\CSVs\Feature_engg_Paper2.csv", index = False)